In [ ]:
# TensorFlow 설치
%pip install tensorflow

# PyTorch 설치 (기본 CPU 버전)
%pip install torch torchvision torchaudio

# PyTorch로 만든 모델을 scikit-learn처럼 편하게 학습/평가/튜닝할 때 사용
%pip install skorch
# TensorFlow 설치
%pip install tensorflow

# PyTorch 설치 (기본 CPU 버전)
# %pip install torch torchvision torchaudio

# NumPy, Pandas, scikit-learn 설치
%pip install numpy pandas scikit-learn

In [ ]:
%pip install matplotlib

In [ ]:
%pip uninstall torch torchvision torchaudio -y

In [ ]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu130


In [ ]:
import torch
print(torch.version.cuda)          # CUDA 버전 확인
print(torch.cuda.is_available())   # True = GPU 사용 가능
print(torch.cuda.get_device_name(0)) # GPU 이름 출력


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

#### 데이터 불러오기

In [ ]:
# # 데이터 불러오기 1 : 기존 수치형 데이터
# import pandas as pd
# url = "C:\\Users\\Playdata2\\Downloads\\tree_model_preprocessed.csv"
# df = pd.read_csv(url)
# df.head()

In [1]:
# 데이터 불러오기 2 : 로그변환 수치형 데이터
import pandas as pd
url = "../../DL_data/dataset/re_log_model_preprocessed.csv"
df = pd.read_csv(url)
df.head()

,subscription_age,service_failure_count,download_over_limit,churn,contract_type_active,contract_type_expired,contract_type_no_contract,sub_both,sub_movie,sub_none,sub_tv,bill_avg_log,download_avg_log,upload_avg_log
0,11.95,0,0,0,1,0,0,0,0,0,1,3.258097,2.240710,1.193922
1,8.22,0,0,1,0,0,1,0,0,1,0,0.000000,0.000000,0.000000
2,8.91,0,0,1,0,1,0,0,0,0,1,2.833213,2.687847,0.641854
3,6.87,1,0,1,0,0,1,0,0,1,0,3.091042,0.000000,0.000000
4,6.39,0,0,1,0,0,1,0,0,1,0,0.000000,0.000000,0.000000


In [ ]:
# 랜덤서치 3-150
import torch
import torch.nn as nn
import torch.optim as optim
from skorch import NeuralNetClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import numpy as np
from skorch.callbacks import EarlyStopping,EpochScoring

# GPU 자동 선택
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

# 데이터 준비
X = df.drop('churn', axis=1).values.astype(np.float32)
y = df['churn'].values.astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# PyTorch 모델 정의 (BCEWithLogitsLoss용)
class ChurnModel(nn.Module):
    def __init__(self, layers=[64,32,16], activation=nn.ReLU):
        super().__init__()
        layer_list = []
        input_dim = X_train.shape[1]
        for l in layers:
            layer_list.append(nn.Linear(input_dim, l))
            layer_list.append(activation())
            input_dim = l
        layer_list.append(nn.Linear(input_dim, 1))  # 마지막 Linear
        self.network = nn.Sequential(*layer_list)
    
    def forward(self, X):
        return self.network(X).squeeze(1)

# skorch 래퍼
net = NeuralNetClassifier(
    module=ChurnModel,
    max_epochs=10,
    lr=0.001,
    batch_size=32,
    optimizer=optim.Adam,
    criterion=nn.BCEWithLogitsLoss,
    iterator_train__shuffle=True,
    device=device,
    callbacks=[EarlyStopping(patience=5), # 개선 없으면 학습 중단
        EpochScoring('accuracy', on_train=True, name='train_acc'),
        EpochScoring('accuracy', on_train=False, name='valid_acc')
    ],
    verbose=1,
)

# 파이프라인 생성
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('net', net)
])

# 랜덤서치할 파라미터
param_dist = {
    'net__module__layers': [[16, 8], [16, 8, 4], [16, 8, 16], [16, 32, 16], [64, 32], [64, 32, 16], [64, 32, 64], [128, 64, 32]],
    'net__module__activation': [nn.ReLU, nn.Tanh],
    'net__optimizer': [optim.Adam, optim.SGD],
    'net__lr': [0.001, 0.01, 0.1],
    'net__optimizer__weight_decay': [0.0, 0.0001, 0.001],
    'net__batch_size': [32, 64, 128],
    'net__max_epochs': [10, 20, 30]
}

random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_dist,
    n_iter=150,
    cv=3,
    n_jobs=-1,
    verbose=1,
    random_state=42
)

# RandomizedSearchCV 학습
random_result = random_search.fit(X_train, y_train)

# 최적 모델로 예측
best_model = random_result.best_estimator_
y_pred = best_model.predict(X_test)

# classification metrics
print("Accuracy: {:.4f}".format(accuracy_score(y_test, y_pred)))
print("Precision: {:.4f}".format(precision_score(y_test, y_pred)))
print("Recall: {:.4f}".format(recall_score(y_test, y_pred)))
print("F1 Score: {:.4f}".format(f1_score(y_test, y_pred)))
print("\nClassification Report:\n", classification_report(y_test, y_pred, digits=4))

# 최적 하이퍼파라미터 확인
print("Best Parameters Found:")
print(random_result.best_params_)


Using device: cuda
Fitting 3 folds for each of 150 candidates, totalling 450 fits
  epoch    train_acc    train_loss    valid_acc    valid_loss     dur
-------  -----------  ------------  -----------  ------------  ------
      1       0.9312        0.2247       0.9350        0.2045  1.6397
      2       0.9339        0.2086       0.9345        0.2027  1.5303
      3       0.9339        0.2055       0.9352        0.1994  1.6467
      4       0.9348        0.2036       0.9363        0.1971  1.7901
      5       0.9349        0.2023       0.9365        0.1950  1.8105
      6       0.9353        0.2014       0.9365        0.1952  1.8300
      7       0.9360        0.2004       0.9377        0.1941  1.8279
      8       0.9362        0.1992       0.9382        0.1949  1.8041
      9       0.9367        0.1988       0.9384        0.1931  1.8049
     10       0.9367        0.1979       0.9382        0.1912  1.8173
     11       0.9369        0.1971       0.9374        0.1910  1.8163
     12 

In [ ]:
# 랜덤서치 3-300
import torch
import torch.nn as nn
import torch.optim as optim
from skorch import NeuralNetClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import numpy as np
from skorch.callbacks import EarlyStopping,EpochScoring

# GPU 자동 선택
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

# 데이터 준비
X = df.drop('churn', axis=1).values.astype(np.float32)
y = df['churn'].values.astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# PyTorch 모델 정의 (BCEWithLogitsLoss용)
class ChurnModel(nn.Module):
    def __init__(self, layers=[64,32,16], activation=nn.ReLU):
        super().__init__()
        layer_list = []
        input_dim = X_train.shape[1]
        for l in layers:
            layer_list.append(nn.Linear(input_dim, l))
            layer_list.append(activation())
            input_dim = l
        layer_list.append(nn.Linear(input_dim, 1))  # 마지막 Linear
        self.network = nn.Sequential(*layer_list)
    
    def forward(self, X):
        return self.network(X).squeeze(1)

# skorch 래퍼
net = NeuralNetClassifier(
    module=ChurnModel,
    max_epochs=10,
    lr=0.001,
    batch_size=32,
    optimizer=optim.Adam,
    criterion=nn.BCEWithLogitsLoss,
    iterator_train__shuffle=True,
    device=device,
    callbacks=[EarlyStopping(patience=5), # 개선 없으면 학습 중단
        EpochScoring('accuracy', on_train=True, name='train_acc'),
        EpochScoring('accuracy', on_train=False, name='valid_acc')
    ],
    verbose=1,
)

# 파이프라인 생성
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('net', net)
])

# 랜덤서치할 파라미터
param_dist = {
    'net__module__layers': [[16, 8], [16, 8, 4], [16, 8, 16], [16, 32, 16], [64, 32], [64, 32, 16], [64, 32, 64], [128, 64, 32]],
    'net__module__activation': [nn.ReLU, nn.Tanh],
    'net__optimizer': [optim.Adam, optim.SGD],
    'net__lr': [0.001, 0.01, 0.1],
    'net__optimizer__weight_decay': [0.0, 0.0001, 0.001],
    'net__batch_size': [32, 64, 128],
    'net__max_epochs': [10, 20, 30]
}

random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_dist,
    n_iter= 300,
    cv=3,
    n_jobs=-1,
    verbose=1,
    random_state=42
)

# RandomizedSearchCV 학습
random_result = random_search.fit(X_train, y_train)

# 최적 모델로 예측
best_model = random_result.best_estimator_
y_pred = best_model.predict(X_test)

# classification metrics
print("Accuracy: {:.4f}".format(accuracy_score(y_test, y_pred)))
print("Precision: {:.4f}".format(precision_score(y_test, y_pred)))
print("Recall: {:.4f}".format(recall_score(y_test, y_pred)))
print("F1 Score: {:.4f}".format(f1_score(y_test, y_pred)))
print("\nClassification Report:\n", classification_report(y_test, y_pred, digits=4))

# 최적 하이퍼파라미터 확인
print("Best Parameters Found:")
print(random_result.best_params_)


Using device: cuda
Fitting 3 folds for each of 300 candidates, totalling 900 fits
  epoch    train_acc    train_loss    valid_acc    valid_loss     dur
-------  -----------  ------------  -----------  ------------  ------
      1       0.9212        0.2531       0.9352        0.2024  0.9982
      2       0.9341        0.2063       0.9365        0.1998  1.0180
      3       0.9348        0.2038       0.9371        0.1983  1.0128
      4       0.9351        0.2025       0.9368        0.1983  1.0089
      5       0.9355        0.2014       0.9371        0.1967  1.0188
      6       0.9358        0.2010       0.9371        0.1969  1.0305
      7       0.9358        0.2008       0.9373        0.1949  1.0179
      8       0.9356        0.2003       0.9366        0.1955  1.0047
      9       0.9362        0.1997       0.9365        0.1968  1.0390
     10       0.9362        0.1992       0.9373        0.1941  1.0367
     11       0.9361        0.1984       0.9368        0.1941  1.0094
     12 

In [ ]:
# 랜덤서치 2-600
import torch
import torch.nn as nn
import torch.optim as optim
from skorch import NeuralNetClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import numpy as np
from skorch.callbacks import EarlyStopping,EpochScoring

# GPU 자동 선택
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

# 데이터 준비
X = df.drop('churn', axis=1).values.astype(np.float32)
y = df['churn'].values.astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# PyTorch 모델 정의 (BCEWithLogitsLoss용)
class ChurnModel(nn.Module):
    def __init__(self, layers=[64,32,16], activation=nn.ReLU):
        super().__init__()
        layer_list = []
        input_dim = X_train.shape[1]
        for l in layers:
            layer_list.append(nn.Linear(input_dim, l))
            layer_list.append(activation())
            input_dim = l
        layer_list.append(nn.Linear(input_dim, 1))  # 마지막 Linear
        self.network = nn.Sequential(*layer_list)
    
    def forward(self, X):
        return self.network(X).squeeze(1)

# skorch 래퍼
net = NeuralNetClassifier(
    module=ChurnModel,
    max_epochs=10,
    lr=0.001,
    batch_size=32,
    optimizer=optim.Adam,
    criterion=nn.BCEWithLogitsLoss,
    iterator_train__shuffle=True,
    device=device,
    callbacks=[EarlyStopping(patience=5), # 개선 없으면 학습 중단
        EpochScoring('accuracy', on_train=True, name='train_acc'),
        EpochScoring('accuracy', on_train=False, name='valid_acc')
    ],
    verbose=1,
)

# 파이프라인 생성
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('net', net)
])

# 랜덤서치할 파라미터
param_dist = {
    'net__module__layers': [[16, 8], [16, 8, 4], [16, 8, 16], [16, 32, 16], [64, 32], [64, 32, 16], [64, 32, 64], [128, 64, 32]],
    'net__module__activation': [nn.ReLU, nn.Tanh],
    'net__optimizer': [optim.Adam, optim.SGD],
    'net__lr': [0.001, 0.01, 0.1],
    'net__optimizer__weight_decay': [0.0, 0.0001, 0.001],
    'net__batch_size': [256, 64, 128],
    'net__max_epochs': [10, 20, 30]
}

random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_dist,
    n_iter= 600,
    cv=2,
    n_jobs=-1,
    verbose=1,
    random_state=42
)

# RandomizedSearchCV 학습
random_result = random_search.fit(X_train, y_train)

# 최적 모델로 예측
best_model = random_result.best_estimator_
y_pred = best_model.predict(X_test)

# classification metrics
print("Accuracy: {:.4f}".format(accuracy_score(y_test, y_pred)))
print("Precision: {:.4f}".format(precision_score(y_test, y_pred)))
print("Recall: {:.4f}".format(recall_score(y_test, y_pred)))
print("F1 Score: {:.4f}".format(f1_score(y_test, y_pred)))
print("\nClassification Report:\n", classification_report(y_test, y_pred, digits=4))

# 최적 하이퍼파라미터 확인
print("Best Parameters Found:")
print(random_result.best_params_)


Using device: cuda
Fitting 2 folds for each of 600 candidates, totalling 1200 fits
  epoch    train_acc    train_loss    valid_acc    valid_loss     dur
-------  -----------  ------------  -----------  ------------  ------
      1       0.9275        0.2238       0.9356        0.2006  1.5231
      2       0.9341        0.2059       0.9350        0.1998  1.4516
      3       0.9344        0.2054       0.9362        0.1970  1.3742
      4       0.9353        0.2037       0.9361        0.1948  1.3873
      5       0.9356        0.2022       0.9369        0.1955  1.3822
      6       0.9356        0.2015       0.9371        0.1964  1.4182
      7       0.9365        0.2001       0.9359        0.1970  1.3885
      8       0.9356        0.2003       0.9365        0.1962  1.3784
Stopping since valid_loss has not improved in the last 5 epochs.
Accuracy: 0.9350
Precision: 0.9529
Recall: 0.9298
F1 Score: 0.9412

Classification Report:
               precision    recall  f1-score   support

     

In [ ]:

# 랜덤서치 2 150
import torch
import torch.nn as nn
import torch.optim as optim
from skorch import NeuralNetClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import numpy as np
from skorch.callbacks import EarlyStopping,EpochScoring

# GPU 자동 선택
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

# 데이터 준비
X = df.drop('churn', axis=1).values.astype(np.float32)
y = df['churn'].values.astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# PyTorch 모델 정의 (BCEWithLogitsLoss용)
class ChurnModel(nn.Module):
    def __init__(self, layers=[64,32,16], activation=nn.ReLU):
        super().__init__()
        layer_list = []
        input_dim = X_train.shape[1]
        for l in layers:
            layer_list.append(nn.Linear(input_dim, l))
            layer_list.append(activation())
            input_dim = l
        layer_list.append(nn.Linear(input_dim, 1))  # 마지막 Linear
        self.network = nn.Sequential(*layer_list)
    
    def forward(self, X):
        return self.network(X).squeeze(1)

# skorch net
net = NeuralNetClassifier(
    module=ChurnModel,
    max_epochs=20,         # 최대 epoch 줄임
    lr=0.001,
    batch_size=64,         # 배치 크기 늘림
    optimizer=optim.Adam,
    criterion=nn.BCEWithLogitsLoss,
    iterator_train__shuffle=True,
    device=device,
    callbacks=[EarlyStopping(patience=5), # 개선 없으면 학습 중단
        EpochScoring('accuracy', on_train=True, name='train_acc'),
        EpochScoring('accuracy', on_train=False, name='valid_acc')
    ],
    verbose=1,
)

# 파이프라인 생성
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('net', net)
])

# 랜덤서치할 파라미터
param_dist = {
    'net__module__layers': [[16, 8], [16, 8, 4], [16, 8, 16], [16, 32, 16], [64, 32], [64, 32, 16], [64, 32, 64], [128, 64, 32]],
    'net__module__activation': [nn.ReLU, nn.Tanh],
    'net__optimizer': [optim.Adam, optim.SGD],
    'net__lr': [0.001, 0.01, 0.1],
    'net__optimizer__weight_decay': [0.0, 0.0001, 0.001],
    'net__batch_size': [256, 64, 128],
    'net__max_epochs': [10, 20, 30]
}

# RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_dist,
    n_iter=150,           # 500 -> 150으로 줄임
    cv=2,                 # 3-fold -> 2-fold
    n_jobs=-1,
    verbose=1,
    random_state=42
)

# RandomizedSearchCV 학습
random_result = random_search.fit(X_train, y_train)

# 최적 모델로 예측
best_model = random_result.best_estimator_
y_pred = best_model.predict(X_test)

# classification metrics
print("Accuracy: {:.4f}".format(accuracy_score(y_test, y_pred)))
print("Precision: {:.4f}".format(precision_score(y_test, y_pred)))
print("Recall: {:.4f}".format(recall_score(y_test, y_pred)))
print("F1 Score: {:.4f}".format(f1_score(y_test, y_pred)))
print("\nClassification Report:\n", classification_report(y_test, y_pred, digits=4))

# 최적 하이퍼파라미터 확인
print("Best Parameters Found:")
print(random_result.best_params_)

Using device: cuda
Fitting 2 folds for each of 150 candidates, totalling 300 fits


c:\Users\USER\Desktop\프로젝트2\.venv\Lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
286 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\USER\Desktop\프로젝트2\.venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\USER\Desktop\프로젝트2\.venv\Lib\site-packages\sklearn\base.py", line 1365, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\USER\Desktop\프로젝트2\.venv\Lib\site-packages\sklearn\pipeline.py", line 663,

  epoch    train_acc    train_loss    valid_acc    valid_loss     dur
-------  -----------  ------------  -----------  ------------  ------
      1       0.9307        0.2192       0.9356        0.2000  0.9607
      2       0.9338        0.2083       0.9363        0.1990  0.8904
      3       0.9339        0.2067       0.9359        0.2000  0.9135
      4       0.9344        0.2058       0.9352        0.2013  0.9010
      5       0.9344        0.2063       0.9346        0.1998  0.9279
      6       0.9341        0.2048       0.9371        0.1980  0.9048
      7       0.9343        0.2055       0.9350        0.2040  0.9126
      8       0.9346        0.2042       0.9370        0.1995  0.9302
      9       0.9345        0.2042       0.9368        0.1955  0.8867
     10       0.9348        0.2036       0.9366        0.1990  0.9257
Accuracy: 0.9337
Precision: 0.9473
Recall: 0.9334
F1 Score: 0.9403

Classification Report:
               precision    recall  f1-score   support

         0.0 

In [ ]:
# 시각화
import matplotlib.pyplot as plt
# best_model 가져오기
best_model = random_result.best_estimator_

# skorch history 접근
history = best_model.named_steps['net'].history

# epoch별 train/valid loss, accuracy
train_loss = [h['train_loss'] for h in history]
valid_loss = [h['valid_loss'] for h in history]
train_acc = [h['train_acc'] for h in history if 'train_acc' in h]
valid_acc = [h['valid_acc'] for h in history if 'valid_acc' in h]
epochs = range(1, len(train_loss)+1)

# 손실값 시각화
plt.figure(figsize=(10,5))
plt.plot(epochs, train_loss, label='Train Loss')
plt.plot(epochs, valid_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training & Validation Loss')
plt.legend()
plt.show()

# 정확도 시각화
if train_acc and valid_acc:
    plt.figure(figsize=(10,5))
    plt.plot(epochs, train_acc, label='Train Accuracy')
    plt.plot(epochs, valid_acc, label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training & Validation Accuracy')
    plt.legend()
    plt.show()